# Welcome to TQ42

## Example of Long Short-Term Memory (LSTM)

In [23]:
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto
from tq42.algorithm import (
    AlgorithmProto, 
    OptimProto, 
    LossFuncProto, 
    DatasetStorageInfoProto, 
    TSLSTMTrainMetadataProto, 
    TSLSTMTrainParametersProto,
    MLTrainInputsProto
)
from google.protobuf.json_format import MessageToDict

# Create a client and selecting org, proj and exp

In [24]:

with TQ42Client() as client:
    org_sample = list_all_organizations(client=client)[0] 
    proj_sample = list_all_projects(client=client, organization_id=org_sample.id)[0]
    exp_sample = list_all_experiments(client=client, project_id=proj_sample.id)[-1]

    print(proj_sample)
    print(org_sample)
    print(exp_sample)



# Set parameters for the train

In [25]:

parameters_train = MessageToDict(TSLSTMTrainMetadataProto(
    parameters=TSLSTMTrainParametersProto(
        input_width=24,
        label_width=1,
        hidden_size=17,
        dropout_coef=0.17,
        num_epochs=20,
        batch_size=128,
        learning_rate=0.001,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MAE,
        time_column="Time",
        target_column="Power, kW"
    ),
    inputs=MLTrainInputsProto(
        data=DatasetStorageInfoProto(storage_id="fc4310c5-a2a3-43cd-990c-8dd2b1aec994")
    )
), preserving_proto_field_name=True)

# Train the model

In [26]:

run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.TS_LSTM_TRAIN, 
    experiment_id=exp_sample.id,
    compute=HardwareProto.SMALL, 
    parameters=parameters_train
)

print(run.data)

id: "e4724c5b-414d-4c31-9181-b0b7aaaab9ef"
experiment_id: "811fd59b-2b51-42ea-b573-b5aebb0d5d7a"
sequential_id: 13
status: QUEUED
algorithm: TS_LSTM_TRAIN
hardware: SMALL
metadata: "{\"parameters\":{\"input_width\":24,\"label_width\":1,\"hidden_size\":17,\"dropout_coef\":0.17,\"num_epochs\":20,\"batch_size\":128,\"learning_rate\":0.001,\"optim\":\"ADAM\",\"loss_func\":\"MAE\",\"train_model_info\":null},\"inputs\":{\"data\":{\"storage_id\":\"fc4310c5-a2a3-43cd-990c-8dd2b1aec994\"}}}"
result {
}
created_by: "35dc8cba-b06d-4f15-b9bc-588f608789de"
created_at {
  seconds: 1705660472
  nanos: 338705048
}


# Import functions for the evaluate

In [27]:
from tq42.algorithm import (
    ModelStorageInfoProto,
    TSEvalInputsProto,
    TSLSTMEvalMetadataProto,
    TSLSTMEvalParametersProto
)

# Set parameters for the evaluate

In [28]:
params_eval = MessageToDict(TSLSTMEvalMetadataProto(
    parameters=TSLSTMEvalParametersProto(
        input_width=1,
        label_width=1,
        hidden_size=17,
        dropout_coef=0.24,
        time_column="Time"
    ),
    inputs=TSEvalInputsProto(
        model=ModelStorageInfoProto(storage_id="MODEL_BUCKET_STORAGE_ID"),
        data=DatasetStorageInfoProto(storage_id="DATA_BUCKET_STORAGE_ID")
    )
), preserving_proto_field_name=True)

# Evaluate the model

In [29]:
    run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.TS_LSTM_EVAL,
        experiment_id=exp_sample.id,
        compute=HardwareProto.SMALL,
        parameters=params_eval
    )
print(run.data)

id: "ff12f5d8-ee02-4438-a138-ec4969d2d361"
experiment_id: "811fd59b-2b51-42ea-b573-b5aebb0d5d7a"
sequential_id: 14
status: QUEUED
algorithm: TS_LSTM_EVAL
hardware: SMALL
metadata: "{\"parameters\":{\"input_width\":1,\"label_width\":1,\"hidden_size\":17,\"dropout_coef\":0.24},\"inputs\":{\"model\":{\"storage_id\":\"MODEL_BUCKET_STORAGE_ID\"},\"data\":{\"storage_id\":\"DATA_BUCKET_STORAGE_ID\"}}}"
result {
}
created_by: "35dc8cba-b06d-4f15-b9bc-588f608789de"
created_at {
  seconds: 1705660479
  nanos: 305211590
}
